In [2]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view as windows
import pandas as pd
from einops import rearrange, repeat
from datetime import timedelta
from datetime import datetime
from tqdm import tqdm
from numpy import ma

In [ ]:
df = pd.read_csv('data/Hunter_U2_5minData_MASTER.csv').replace(-99999.000000, np.NaN).dropna()
manipcols = pd.read_csv('data/manip_cols.csv')
inputcols = pd.read_csv('data/input_cols.csv')
target = ['KLBH_PER_MW_ONTIME']
manip_cols = [row[1].upper() for row in manipcols.values]
input_cols = [row[1].upper() for row in inputcols.values]
df.columns = df.columns.str.upper()
df = df.loc[:, input_cols]
df.describe()

In [ ]:
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce')
df["HOUR"] = df["DATE_TIME"].dt.hour
df.insert(0, 'HOUR', df.pop('HOUR'))
df["DAY"] = df["DATE_TIME"].dt.day
df.insert(0, 'DAY', df.pop('DAY'))
df["MONTH"] = df["DATE_TIME"].dt.month
df.insert(0, 'MONTH', df.pop('MONTH'))
df.index = pd.DatetimeIndex(df.pop('DATE_TIME'))
print(df.shape)
df.head()

In [ ]:
idxs = df.index
td = timedelta(minutes=5)
m1 = timedelta(minutes=1)
maxstep = timedelta(minutes=6)
addnans = [(i, ((idxs[i+1] - idxs[i])//td)-1) for i in tqdm(range(len(idxs)-1)) if idxs[i+1] - idxs[i] > maxstep]
print(len(addnans))
print('addnans', addnans)

In [ ]:
idxs2 = []
idxs3 = []
for i, l in addnans:
    idx = idxs[i]
    if l < 4:
        for j in range(l):
            idx += td
            idxs2.append(idx)
    else:
        for j in range(l):
            idx += td
            idxs3.append(idx)

print(len(idxs2), len(idxs3), sum((len(idxs2), len(idxs3))))

In [ ]:
repline2 = -99999.000000 * np.ones((len(idxs2), df.shape[1]))
df2 = pd.DataFrame(repline2, columns=df.columns, index=idxs2)
repline3 = -99999.000000 * np.ones((len(idxs3), df.shape[1]))
df3 = pd.DataFrame(repline3, columns=df.columns, index=idxs3)
print(df2.shape, df3.shape)
df2.head()

In [ ]:
df = pd.concat([df, df2]).sort_index().replace(-99999.000000, np.NaN)
df = df.interpolate(method='linear')
df = pd.concat([df, df3]).sort_index().replace(-99999.000000, np.NaN)
del df2, df3, addnans
print(df.shape)
df.head()

In [ ]:
df = df.reset_index(drop=True)
df_fitter = df.loc[df['KLBH_PER_MW_ONTIME'] < 1.2]
df_fitter = df_fitter.loc[df_fitter['KLBH_PER_MW_ONTIME'] > 0.8].dropna()
mins = df_fitter.min()
maxs = df_fitter.max()

df = (df-mins)/(maxs-mins)
mins.to_csv('data/minsU2.csv')
maxs.to_csv('data/maxsU2.csv')
del mins
del maxs

df.to_csv('data/scaled_U2_data.csv')
df.describe()

In [2]:
df = pd.read_csv('data/scaled_U2_data.csv')
x1 = np.array(df)
del df
x = rearrange(windows(x1, [22, x1.shape[1]], axis=[0, 1]), 'b 1 h w -> b h w')

In [6]:
mask = np.array([np.all(np.all(x[xi, :, -1] <= 1) >= 0) and np.all(x[xi, :, :] == x[xi, :, :]) for xi in range(x.shape[0])])
mask = repeat(mask, 'a -> a b c', b=22, c=249)

(220952, 22, 249)
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [9]:
print(x.shape)
x2 = ma.masked_array(x, mask)
print(x2.shape)

(220952, 22, 249)
(220952, 22, 249)


In [10]:
print(np.shares_memory(x, x1))
print(np.shares_memory(x, x2))
print(np.shares_memory(x1, x2))

True
True
True


In [3]:
df = pd.read_csv('data/scaled_U2_data.csv')
df = df.rolling(22, 22, )


In [6]:
print(print(df.ndim))

2
None
